<a href="https://colab.research.google.com/github/trismegisto78/romeo/blob/master/testBidirLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
from random import random
from numpy import array
from numpy import cumsum
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional

Using TensorFlow backend.


In [0]:
!git clone https://github.com/trismegisto78/romeo

fatal: destination path 'romeo' already exists and is not an empty directory.


MOUNT GOOGLE DRIVE

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
with open('/content/gdrive/My Drive/strMacd2004.csv', 'r') as f:
  lerighe = f.read().split('\n')
del lerighe[0]

In [67]:
righe_file=[]
for k in range(0,100000) :
  righe_file.append(float(lerighe[k].split(";")[1]) )
  
print(righe_file[0])

1.2596


In [0]:
n_timesteps = 100

model = Sequential()
model.add(Bidirectional(LSTM(n_timesteps, return_sequences=True), input_shape=(n_timesteps, 1)))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [78]:
ilMax = max(righe_file)
ilMin = min(righe_file)


delta = ilMax-ilMin


print('MAX ', ilMax )
print('MIN ', ilMin )
print('delta ', delta )

MAX  1.2924
MIN  1.1983
delta  0.09410000000000007


In [0]:




# create a sequence classification instance
def get_sequence():
  X = np.zeros(n_timesteps)
  Y = np.zeros(n_timesteps)
  for k in range(0,n_timesteps):
    value=(float(righe_file[samplePos+k])-ilMin )/delta
    value2=(float(righe_file[samplePos+k+1])-ilMin)/delta
    #print(value,' ',value2)
    X[k]=value
    Y[k]=value2
    
    if value2<value:
      Y[k]=1.0
    else:
      Y[k]=0.0
  
  X = X.reshape(1, n_timesteps, 1)
  y = Y.reshape(1, n_timesteps, 1)
  
  return X, y

In [0]:
samplePos=0
# train LSTM
for epoch in range(10000):
  X,y = get_sequence()
  
  model.fit(X, y, epochs=1, batch_size=1, verbose=0)
  if samplePos>4000000 :
    samplePos=0
  samplePos=samplePos+1
  
# evaluate LSTM



In [0]:
samplePos=20000+n_timesteps
X,y = get_sequence()

yhat = model.predict_classes(X, verbose=0)

valH =0
valX=0
for i in range(n_timesteps):
  valx = (yhat[0, i]*delta)+ilMin
  #print('Expected:', '{0:.5f}'.format(float( (y[0, i]*delta)+ilMin ) ), 'Predicted', '{0:.5f}'.format(float(valx)   )   )
  #print('{0:.5f}'.format(float( (y[0, i]*delta)+ilMin ) ), '\t', '{0:.5f}'.format(float(valx)   )   )
  
  
  theH=int(y[0, i])
  theX=int(yhat[0, i])
  
  if theH==1 :
    valH=valH+1
  else:
    valH=valH-1
    
    
  if theX==1 :
    valX=valX+1
  else:
    valX=valX-1
    
  print( valH, '\t',  valX )

In [0]:
samplePos=samplePos+500